In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Input-Output Study t-bin comparison
Our [standard I/O waveset](./standard_results.ipynb) integrates over the entire four-momentum transfer region $0.1\leq -t \leq 1.0$. This is *mostly* a safe bet, as the MC was not generated with any $-t$ dependence, aside from an exponential decay that reduces statistics over the $-t$ range. The caveat is that the acceptance is not necessarily uniform across $-t$, and so the results are subject to change. Here we've performed a fit in just the $0.1\leq -t \leq 0.2$ region to compare with our main result. The waveset remains the same 
* Generated with $b_1(1235)$ and $\rho(1450)$ Breit-Wigners fixed to their PDG values
  * an isotropic background is included, but is so small it's negligible
  * These fit results are from mass-independent fits (no Breit-Wigners) to the signal generated by these BW's.
* No `OmegaDalitz` amplitudes for changing the dalitz distribution of the $\omega$ and its corresponding $\lambda$ distribution
* No $D/S$ ratio that would be typically associated with the $b_1$
* Only in the PARA 0 orientation

Each bin of mass has 500 randomized fits and 100 bootstrap fits 

Note that due to the reduction in statistics, a few more bins failed in this case

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager
import neutralb1.analysis.statistics as stats

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/io-tests/spin-1/t_0.1-0.2/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in preprocessed results
with open(f"{STUDY_DIR}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
    data = pkl.load(f)
    results = ResultManager(**data)

In [ ]:
results.summary()

## Analysis

### Standard Plots

In [ ]:
results.plot.intensity.jp()
plt.show()

In [ ]:
results.plot.intensity.waves()
plt.show()

In [ ]:
results.plot.intensity.waves(fractional=True)
plt.show()

In [ ]:
results.plot.intensity.waves(reflectivity="negative")
plt.savefig(f"{STUDY_DIR}/plots/waves_negative.pdf")

In [ ]:
results.plot.diagnostic.matrix()
plt.show()

In [ ]:
sig_moments = list(results.get_significant_moments(threshold=0.02))
results.plot.intensity.moments(moments=sig_moments)
plt.savefig(f"{STUDY_DIR}/plots/significant_moments.pdf")

### Naturalities

In [ ]:
results.plot.intensity.plot(
    ["p", "m"], 
    fractional=True, 
    col_kwargs={"p": {"color":"red", "label":"Natural"}, "m": {"color":"blue", "label":"Unnatural"}})
plt.show()

### b1 and rho(1450) interference

In [ ]:
colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot
results.plot.phase.mass_phase(
    "p1p0S", "p1mpP", 
    amp1_kwargs={"color": colors[2]},
    amp2_kwargs={"color": colors[3]},
    )
plt.savefig(f"{STUDY_DIR}/plots/interference.pdf")

## Comparison to Previous Results
All previous plots were just to log what the results looked like standalone, but now we can compare them to the results over the full t range. Since the total intensity will have changed, we need to compare fit fractions

In [ ]:
with open(f"{STUDY_DIR}/../t_0.1-1.0/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
    data_full = pkl.load(f)
    results_full = ResultManager(**data_full)

### Moments

In [ ]:
assert results.proj_moments_df is not None
moments = [c for c in results.proj_moments_df.columns if c.startswith("H")]
n_moments = len(moments)
fig, axs = plt.subplots(
    nrows=int(np.ceil(n_moments / 3)),
    ncols=min(3, n_moments),
    figsize=(15, 5 * np.ceil(n_moments / 3)),
    sharex=True,    
    squeeze=False,
    layout="constrained"
)

for ax, col in zip(axs.flatten(), moments):
    results.plot.intensity.plot(
        [col], fractional=True, ax=ax,
        col_kwargs={col: {"markeredgecolor":"black", "marker":"o", "markerfacecolor":"white", "label":r"$0.1\leq -t \leq 0.2$"}}
    )
    results_full.plot.intensity.plot(
        [col], fractional=True, ax=ax,
        col_kwargs={col: {"color":"green", "marker":"o", "label":r"$0.1\leq -t \leq 1.0$"}}
    )    
    ax.set_title(utils.convert_moment_name(col))
    ax.ticklabel_format(axis="y", style="plain")
    ax.legend()

#### Significant Moments
Lets just do the significant ones now to present

In [ ]:
sig_moments = results.get_significant_moments(threshold=0.02).union(results_full.get_significant_moments(threshold=0.02))
sig_moments.remove("H0_0000") # always 1 for fit fractions

n_moments = len(sig_moments)
fig, axs = plt.subplots(
    nrows=int(np.ceil(n_moments / 4)),
    ncols=min(4, n_moments),
    figsize=(15, 5 * np.ceil(n_moments / 4)),
    sharex=True,    
    squeeze=False,
    layout="constrained"
)

for i, (ax, col) in enumerate(zip(axs.flatten(), sig_moments)):
    results.plot.intensity.plot(
        [col], fractional=True, ax=ax,
        col_kwargs={col: {"color":"black", "markeredgecolor":"black", "marker":"o", "markerfacecolor":"white", "label":r"$0.1\leq -t \leq 0.2$", "alpha": 0.7}}
    )
    results_full.plot.intensity.plot(
        [col], fractional=True, ax=ax,
        col_kwargs={col: {"color":"green", "marker":"o", "label":r"$0.1\leq -t \leq 1.0$", "alpha": 0.7}}
    )    
    ax.ticklabel_format(axis="y", style="plain")
    ax.set_title(utils.convert_moment_name(col))
    ax.legend()
    if i != 0:
        ax.get_legend().remove()
    if i % 4 != 0:
        ax.set_ylabel("")

plt.savefig(f"{STUDY_DIR}/plots/significant_moments_comparison.pdf")

### All Amplitudes

In [ ]:
amps = results.coherent_sums["eJPmL"]
n_amps = len(amps)

fig, axs = plt.subplots(
    nrows=int(np.ceil(n_amps / 4)),
    ncols=min(4, n_amps),
    figsize=(15, 5 * np.ceil(n_amps / 4)),
    sharex=True,
    squeeze=False,
    layout="constrained"
)

for i, (ax, amp) in enumerate(zip(axs.flatten(), amps)):
    results.plot.intensity.plot(
        [amp], fractional=True, ax=ax,
        col_kwargs={amp: {"color":"black", "markeredgecolor":"black", "marker":"o", "markerfacecolor":"white", "label":r"$0.1\leq -t \leq 0.2$", "alpha": 0.7}}
    )
    results_full.plot.intensity.plot(
        [amp], fractional=True, ax=ax,
        col_kwargs={amp: {"color":"green", "marker":"o", "label":r"$0.1\leq -t \leq 1.0$", "alpha": 0.7}}
    )    
    ax.ticklabel_format(axis="y", style="plain")
    ax.set_title(utils.convert_amp_name(amp))
    ax.legend()
    if i != 0:
        ax.get_legend().remove()
    if i % 4 != 0:
        ax.set_ylabel("")

for ax in axs.flatten()[n_amps:]:
    ax.set_visible(False)

#### Significant Amplitudes

In [ ]:
sig_amps = results.get_significant_amplitudes(threshold=0.02).union(results_full.get_significant_amplitudes(threshold=0.02))
n_amps = len(sig_amps)
fig, axs = plt.subplots(
    nrows=int(np.ceil(n_amps / 4)),
    ncols=min(4, n_amps),
    figsize=(15, 5 * np.ceil(n_amps / 4)),
    sharex=True,
    squeeze=False,
    layout="constrained"
)

for i, (ax, amp) in enumerate(zip(axs.flatten(), sig_amps)):
    results.plot.intensity.plot(
        [amp], fractional=True, ax=ax,
        col_kwargs={amp: {"color":"black", "markeredgecolor":"black", "marker":"o", "markerfacecolor":"white", "label":r"$0.1\leq -t \leq 0.2$", "alpha": 0.7}}
    )
    results_full.plot.intensity.plot(
        [amp], fractional=True, ax=ax,
        col_kwargs={amp: {"color":"green", "marker":"o", "label":r"$0.1\leq -t \leq 1.0$", "alpha": 0.7}}
    )    
    ax.ticklabel_format(axis="y", style="plain")
    ax.set_title(utils.convert_amp_name(amp))
    ax.legend()
    if i != 0:
        ax.get_legend().remove()
    if i % 4 != 0:
        ax.set_ylabel("")

for ax in axs.flatten()[n_amps:]:
    ax.set_visible(False)

plt.savefig(f"{STUDY_DIR}/plots/significant_amplitudes_comparison.pdf")

### Phases
Since the unnatural phases are heavily dominated by error and there are tons of natural phases we could choose, we'll simply choose phases between the most significant amplitudes (using a 2% threshold)

In [ ]:
phases = results.get_significant_phases(threshold=0.02).union(results_full.get_significant_phases(threshold=0.02))
n_phases = len(phases)

fig, axs = plt.subplots(
    nrows=int(np.ceil(n_phases / 4)),
    ncols=min(4, n_phases),
    figsize=(15, 5 * np.ceil(n_phases / 4)),
    sharex=True,    
    squeeze=False,
    layout="constrained"
)
for i, (ax, phase) in enumerate(zip(axs.flatten(), phases)):
    amp1, amp2 = phase.split("_")
    results.plot.phase.phase(
        amp1, amp2, ax=ax,
        phase_kwargs={"color":"black", "markeredgecolor":"black", "marker":"o", "markerfacecolor":"white", "label":r"$0.1\leq -t \leq 0.2$", "alpha": 0.7},
        extend_range=False
    )
    results_full.plot.phase.phase(
        amp1, amp2, ax=ax,
        phase_kwargs={"color":"green", "marker":"o", "label":r"$0.1\leq -t \leq 1.0$", "alpha": 0.7},        
        extend_range=False
    )    
    ax.set_title(utils.convert_amp_name(phase))
    ax.legend()
    if i != 0:
        ax.get_legend().remove()
    if i % 4 != 0:
        ax.set_ylabel("")

for ax in axs.flatten()[n_phases:]:
    ax.set_visible(False)

#### Selected Phases
Let's choose a couple well-captures ones to look at in detail

In [ ]:
phases = [
    results.phase_difference_dict[("p1p0S", "p1mpP")],
    results.phase_difference_dict[("p1ppS", "p1mpP")],
    results.phase_difference_dict[("p1p0D", "p1mpP")],
    results.phase_difference_dict[("p1p0D", "p1p0S")],
]

fig, axs = plt.subplots(
    nrows=2,
    ncols=2,
    figsize=(12, 10),
    sharex=True,    
    squeeze=False,
    layout="constrained"
)
for i, (ax, phase) in enumerate(zip(axs.flatten(), phases)):
    amp1, amp2 = phase.split("_")
    results.plot.phase.phase(
        amp1, amp2, ax=ax,
        phase_kwargs={"color":"black", "markeredgecolor":"black", "marker":"o", "markerfacecolor":"white", "label":r"$0.1\leq -t \leq 0.2$", "alpha": 0.7},
        truth_kwargs={"color":"black"},
        extend_range=False
    )
    results_full.plot.phase.phase(
        amp1, amp2, ax=ax,
        phase_kwargs={"color":"green", "marker":"o", "label":r"$0.1\leq -t \leq 1.0$", "alpha": 0.7},        
        truth_kwargs={"color":"green"},
        extend_range=False
    )    
    ax.set_title(utils.convert_amp_name(phase))
    ax.legend()
    if i != 0:
        ax.get_legend().remove()
    if i % 2 != 0:
        ax.set_ylabel("")

plt.savefig(f"{STUDY_DIR}/plots/significant_phases_comparison.pdf")

## Cleanup

In [ ]:
%%bash
cd $STUDY_DIR/plots
zip plots.zip *.pdf